In [1]:
!pip install pytorch-crf

In [1]:
import torch.nn as nn
import torch
torch.manual_seed(20)

In [ ]:
import wandb
wandb_api_key="e62fc492915628e64fcac9c082089ffed84dc72d"
wandb.login(key=wandb_api_key)
run = wandb.init(
    project='rohit_nlp', 
    name='spanbert',
    job_type="training", 
    anonymous="allow",
)

In [2]:
from datasets import load_dataset

# Load SQuAD v2 dataset
dataset = load_dataset("squad_v2")
train_data = dataset["train"]
val_data = dataset["validation"]

In [3]:
print(train_data[0])
question=train_data[0]['question']
context=train_data[0]['context']
answers=train_data[0]['answers']
len(question),len(context),answers,context[269:271]

{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}


(40, 694, {'text': ['in the late 1990s'], 'answer_start': [269]}, 'in')

In [4]:
from transformers import AutoTokenizer
from transformers import AutoModel, AutoTokenizer,BertConfig,BertForQuestionAnswering,BertPreTrainedModel,BertModel
tokenizer=AutoTokenizer.from_pretrained('SpanBERT/spanbert-base-cased')

In [5]:
max_length=384
pad_token=0
def preprocess_train_example(examples):
    examples['question'] = [q.lstrip() for q in examples['question']]

    questions=examples['question']
    answers=examples['answers']
    contexts=examples['context']
    inputs=tokenizer(questions,contexts,
                    truncation='only_second',

                    max_length=max_length,
                    return_offsets_mapping=True,
                    padding='max_length')

    all_answers=[]
    all_labels=[]
    all_offsets=[]
    offset_mappings=inputs.pop('offset_mapping')
    # sample_map = inputs.pop("overflow_to_sample_mapping")
    
    for i,offsets in enumerate(offset_mappings):
        token_types=inputs['token_type_ids'][i]
        example_idx=i
        answer=answers[example_idx]
        all_offsets.append( [
            o if token_types[k] == 1 else None for k, o in enumerate(offsets)
        ])

       
            
        context_start=0
        while token_types[context_start]!=1:
            context_start+=1
        context_end=len(token_types)-1
        while token_types[context_end]!=1:
            context_end-=1
        context_end-=1

        labels=[0 for i in range(context_start)]
        
        if len(answer['text'])==0:
            # labels.append(1)
            labels.extend([0 for i in range(context_end-context_start+1)])
            labels.extend([pad_token for i in range(max_length-context_end-1)])
            all_labels.append(labels)
            all_answers.append('')
            continue
            
        all_answers.append(answer['text'][0])

        start_answer=answer['answer_start'][0]
        end_answer=start_answer+len(answer['text'][0])


        

        if start_answer>offsets[context_end][1]:
            print('here')
            # labels.append(1)
            labels.extend([0 for i in range(context_end-context_start+1)])
            labels.extend([pad_token for i in range(max_length-context_end-1)])
            all_labels.append(labels)
            continue
        
        # print(context_start,context_end)
        
        
        idx=context_start
        while idx<=context_end:
            if offsets[idx][0]>start_answer:
                labels[-1]=1
                idx-=1
                break
            idx+=1
            labels.append(0)
        
        while idx<=context_end:
            if offsets[idx][1]>=end_answer:
                break
            idx+=1
            labels.append(2)##2
        
        while idx<=context_end:
            labels.append(0)
            idx+=1

        while len(labels)<max_length:
            labels.append(pad_token)

        all_labels.append(labels)

    inputs['labels']=all_labels
    inputs['answers']=all_answers
    inputs['offsets']=all_offsets
    inputs['contexts']=contexts

    return inputs




In [6]:
def process_output(context,labels,offsets):
    start=-1
    end=-1
    context_end=0
  
    for i in range(len(labels)):
        if labels[i]==1 and offsets[i]!=None:
            start=i
            end=i
        elif labels[i]==2 and offsets[i]!=None:
            end =i
        # elif labels[i]==0 and input_ids[i]!=tokenizer.pad_token_id:
        #     context_end=i
    # print(context_end,tokenizer.decode(input_ids[context_end]))
    if start==-1:
        return ""
    return context[offsets[start][0]:offsets[end][1]]

In [ ]:
inputs=preprocess_train_example(train_data[32:33])
print(tokenizer.decode(inputs['input_ids'][0][2]))
print()

did



In [7]:
train_dataset=train_data.select(range(15000)).map(preprocess_train_example,batched=True,remove_columns=train_data.column_names)
val_dataset=val_data.map(preprocess_train_example,batched=True,remove_columns=val_data.column_names)
train_dataset.set_format(type='torch')
val_dataset.set_format(type='torch')

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [ ]:
train_data[32]

{'id': '56bf725c3aeaaa14008c9645',
 'title': 'Beyoncé',
 'context': 'A self-described "modern-day feminist", Beyoncé creates songs that are often characterized by themes of love, relationships, and monogamy, as well as female sexuality and empowerment. On stage, her dynamic, highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary popular music. Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a further 60 million with Destiny\'s Child, making her one of the best-selling music artists of all time. She has won 20 Grammy Awards and is the most nominated woman in the award\'s history. The Recording Industry Association of America recognized her as the Top Certified Artist in America during the 2000s decade. In 2009, Billboard named her the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 2011. Time listed her among the 100

In [ ]:
idx=2
print(train_dataset['input_ids'][idx].shape,train_dataset['labels'][idx].shape)
print(train_dataset['answers'][idx],tokenizer.batch_decode(train_dataset['input_ids'][idx]))
print(train_dataset[idx]['contexts'],train_dataset[idx]['answers'])
context=train_dataset[idx]['contexts']
labels=train_dataset[idx]['labels']
offsets=train_dataset[idx]['offsets']
process_output(context,labels,offsets)

torch.Size([384]) torch.Size([384])
2003 ['[CLS]', 'when', 'did', 'be', '##yon', '##ce', 'leave', 'destiny', "'", 's', 'child', 'and', 'become', 'a', 'solo', 'singer', '?', '[SEP]', 'be', '##yon', '##ce', 'g', '##iselle', 'know', '##les', '-', 'cart', '##er', '(', '/', '[UNK]', '/', 'bee', '-', 'yo', '##n', '-', 'say', ')', '(', 'born', 'se', '##pt', '##em', '##ber', '4', ',', '1981', ')', 'is', 'an', 'am', '##eric', '##an', 'singer', ',', 'songwriter', ',', 'record', 'producer', 'and', 'actress', '.', 'born', 'and', 'raised', 'in', 'ho', '##ust', '##on', ',', 'te', '##xa', '##s', ',', 'she', 'performed', 'in', 'various', 'singing', 'and', 'dancing', 'competitions', 'as', 'a', 'child', ',', 'and', 'rose', 'to', 'fame', 'in', 'the', 'late', '1990s', 'as', 'lead', 'singer', 'of', 'r', '&', 'b', 'girl', '-', 'group', 'destiny', "'", 's', 'child', '.', 'managed', 'by', 'her', 'father', ',', 'math', '##ew', 'know', '##les', ',', 'the', 'group', 'became', 'one', 'of', 'the', 'world', "'", 's

'2003'

In [ ]:
from tqdm.auto import tqdm
count=0
for i in tqdm(range(len(train_dataset))):
    context=train_dataset[i]['contexts']
    labels=train_dataset[i]['labels']
    offsets=train_dataset[i]['offsets']
    x=process_output(context,labels,offsets).lower()
    if train_dataset[i]['answers'].lower()!=x:
        count+=1
        # print(train_dataset[i]['answers'],"ans:",x)
print(count)

  0%|          | 0/15000 [00:00<?, ?it/s]

124


In [ ]:
idx=0
print(train_dataset[idx]['answers'])
# print(train_dataset[idx]['start_positions'],train_dataset[idx]['end_positions']+1)
# tokenizer.decode(train_dataset[idx]['input_ids'][train_dataset[idx]['start_positions']:train_dataset[idx]['end_positions']+1])

in the late 1990s


In [8]:
import sys
from typing import List, Optional, Tuple, Union
from transformers import AutoModelForTokenClassification,AutoModelForQuestionAnswering,BertForTokenClassification
from transformers.modeling_outputs import TokenClassifierOutput
from torchcrf import CRF
from torch.nn.utils.rnn import pad_sequence

class CustomConfig(BertConfig):
    def __init__(self,num_labels=3,**kwargs):
        super().__init__(**kwargs)
        self.num_labels=num_labels


class SpanBERT_CRF(BertPreTrainedModel):
    config_class = CustomConfig
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config,add_pooling_layer=False)  # Load SpanBERT
        self.hidden_size = self.bert.config.hidden_size
        self.num_labels = config.num_labels

        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.hidden_size, config.num_labels)  # Fully connected layer

        
        self.crf = CRF(config.num_labels, batch_first=True)  # CRF layer

    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

        emissions = self.fc(sequence_output)  # Convert to logits
        
        if labels is not None:
            class_weights = torch.tensor([1.0, 20.0, 20.0], device=emissions.device).view(1, 1, -1)
            emissions = emissions*class_weights
            # emissions=emissions/emissions.abs().max(dim=-1,keepdim=True)[0]
            loss = -self.crf(emissions, labels, mask=attention_mask.type(torch.uint8), reduction="mean")

            return {"loss": loss, "logits": emissions}
            
        else:
            predictions = self.crf.decode(emissions, mask=attention_mask.byte())
            data_tensors = [torch.tensor(seq) for seq in predictions]

            # Pad with 0 (or another value like -100 for ignored positions)
            predictions = pad_sequence(data_tensors, batch_first=True, padding_value=-100)
            return {"logits": predictions}




class custombert(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            class_weights=torch.tensor([1.0,30.0,30.0]).to(logits.device)
            loss_fct = nn.CrossEntropyLoss(weight=class_weights)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

# model=SpanBERT_CRF.from_pretrained("SpanBERT/spanbert-base-cased",num_labels=3)
model=SpanBERT_CRF.from_pretrained("crf_final",num_labels=3)
# model=custombert.from_pretrained("SpanBERT/spanbert-base-cased",num_labels=3)
# model=SpanBERT_CRF.from_pretrained("./checkpoints/final")

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
from transformers import TrainingArguments, Trainer

def compute_metrics(pred):
    print("here")
    print(pred)

training_args = TrainingArguments(
  output_dir="checkpoints",
  per_device_train_batch_size=40,
  per_device_eval_batch_size=64,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=6,
  learning_rate=1e-5,
  weight_decay=0.01,
  remove_unused_columns=True,
  report_to="wandb",
  load_best_model_at_end=True,
#   no_cuda=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

C:\Users\gupta\AppData\Local\Temp\ipykernel_22876\1390129897.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:530.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch,Training Loss,Validation Loss
1,10.891500,7.970933
2,5.804400,7.603200
3,4.694300,7.530159
4,3.973400,7.652253
5,3.619800,7.798479
6,3.322200,8.065861


TrainOutput(global_step=2250, training_loss=5.384270290798611, metrics={'train_runtime': 5015.0094, 'train_samples_per_second': 17.946, 'train_steps_per_second': 0.449, 'total_flos': 1.763769364992e+16, 'train_loss': 5.384270290798611, 'epoch': 6.0})

In [ ]:
trainer.save_model("./checkpoints/final")

In [ ]:
import gc
# del trainer,model
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
idx=1
batch=train_dataset[idx]
outputs=trainer.predict([batch])
preds=torch.argmax(torch.tensor(outputs.predictions),dim=-1)
process_output(batch['contexts'],preds.tolist()[0],batch['offsets'])

'singing and dancing competitions'

In [10]:
def exact_match_score(predictions, references):
    assert len(predictions) == len(references), "Lists must have the same length"
    matches = sum(p == r for p, r in zip(predictions, references))
    return matches / len(references) * 100 # Convert to percentage

In [ ]:
test_dataset=val_data.map(preprocess_train_example,batched=True,remove_columns=val_data.column_names)
test_dataset.set_format(type='torch')
predictions=trainer.predict(test_dataset).predictions



In [14]:
from tqdm.auto import tqdm
all_preds=[]
for i in tqdm(range(len(predictions))):
    context=test_dataset[i]['contexts']
    preds=torch.argmax(torch.tensor(predictions[i]),dim=-1)
    offsets=test_dataset[i]['offsets']
    all_preds.append(process_output(context,preds,offsets).lower())
answers=test_dataset['answers']
# print(all_preds)
# print(answers)
print(exact_match_score(all_preds,answers))

  0%|          | 0/11873 [00:00<?, ?it/s]

41.75018950560094
